In [3]:
!pip install folium

  Obtaining dependency information for folium from https://files.pythonhosted.org/packages/81/4d/bcbee0676dc06f0b014a030cb928e158c96daafb872a01ddee7e60a8c998/folium-0.19.5-py2.py3-none-any.whl.metadata
  Obtaining dependency information for branca>=0.6.0 from https://files.pythonhosted.org/packages/f8/9d/91cddd38bd00170aad1a4b198c47b4ed716be45c234e09b835af41f4e717/branca-0.8.1-py3-none-any.whl.metadata
  Obtaining dependency information for xyzservices from https://files.pythonhosted.org/packages/9a/6e/49408735dae940a0c1c225c6b908fd83bd6e3f5fae120f865754e72f78cb/xyzservices-2025.1.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/110.9 kB ? eta -:--:--
   --- ------------------------------------ 10.2/110.9 kB ? eta -:--:--
   --- ------------------------------------ 10.2/110.9 kB ? eta -:--:--
   ---------- ---------------------------- 30.7/110.9 kB 262.6 kB/s eta 0:00:01
   --------------------- ----------------- 61.4/110.9 kB 409.6 kB/s eta 0:00:01
   ------

In [8]:
!pip install osmnx

  Obtaining dependency information for osmnx from https://files.pythonhosted.org/packages/77/59/4de56d8db7694831bea788de3a44e196a73f5c1053560b2ccf18282782c8/osmnx-2.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for geopandas>=1.0 from https://files.pythonhosted.org/packages/c4/64/7d344cfcef5efddf9cf32f59af7f855828e9d74b5f862eddf5bfd9f25323/geopandas-1.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for shapely>=2.0 from https://files.pythonhosted.org/packages/27/57/8ec7c62012bed06731f7ee979da7f207bbc4b27feed5f36680b6a70df54f/shapely-2.0.7-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for pyogrio>=0.7.2 from https://files.pythonhosted.org/packages/94/8d/24f21e6a93ca418231aee3bddade7a0766c89c523832f29e08a8860f83e6/pyogrio-0.10.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for pyproj>=3.3.0 from https://files.pythonhosted.org/packages/92/39/ef6f06a5b223dbea308cfcbb7a0f72e7b506aef1850e061b2c73b0818715/pyp

In [9]:
!pip install networkx

In [ ]:
import osmnx as ox
import networkx as nx
import folium

# start and end locations
start_location = (1.3114, 103.8531)  # Farrer Park MRT
end_location = (1.2977, 103.8499)  # SMU

# Fetch road network data within 3km radius around Farrer Park
G = ox.graph_from_point(start_location, dist=3000, network_type="walk", simplify=True)

# Find nearest nodes in the road network
start_node = ox.nearest_nodes(G, start_location[1], start_location[0])
end_node = ox.nearest_nodes(G, end_location[1], end_location[0])

# Route 1: Shortest path using A* algorithm
route1 = nx.astar_path(G, start_node, end_node, weight="length")

# Route 2: Alternative path using different weight (avoiding some main roads)
G_alt = G.copy()
for i in range(len(route1) - 1):
    if G_alt.has_edge(route1[i], route1[i + 1]):
        G_alt.remove_edge(route1[i], route1[i + 1])

# Route 2: Find a new shortest path in the modified graph (Red)
try:
    route2 = nx.astar_path(G_alt, start_node, end_node, weight="length")
except nx.NetworkXNoPath:
    print("No alternative route found! Using a slightly different variation.")
    # If no path exists, use a weighted penalty instead of removal
    for u, v, data in G.edges(data=True):
        if (u, v) in zip(route1, route1[1:]):
            data["length"] *= 2  # Increase weight of first route to force alternative
    route2 = nx.astar_path(G, start_node, end_node, weight="length")

# Convert route nodes to coordinates
route1_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route1]
route2_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route2]

# Create a Folium map
m = folium.Map(location=[1.3045, 103.8515], zoom_start=15)

# Add both routes with different colors
folium.PolyLine(route1_coords, color="blue", weight=6, opacity=0.8, tooltip="Shortest Route").add_to(m)
folium.PolyLine(route2_coords, color="red", weight=6, opacity=0.8, tooltip="Alternative Route").add_to(m)

# Add markers for start and end points
folium.Marker(route1_coords[0], popup="Start: Farrer Park MRT", icon=folium.Icon(color="green", icon="play")).add_to(m)
folium.Marker(route1_coords[-1], popup="End: SMU", icon=folium.Icon(color="red", icon="flag")).add_to(m)

# Save the map as an HTML file
map_path = "farrer_park_to_smu_two_routes.html"
m.save(map_path)

print(f"Map saved!")

No alternative route found! Using a slightly different variation.
Map saved! Open 'farrer_park_to_smu_two_routes.html' in your browser.


In [3]:
!pip install json2html

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for json2html: filename=json2html-1.3.0-py3-none-any.whl size=7694 sha256=6c8f63304638d2e43e73734bb0b6e7a05c807e0a40a472c500a7559094819b12
  Stored in directory: c:\users\99744\appdata\local\pip\cache\wheels\61\94\55\83683c3c00a3f45cb4da4565208d8bd752133bab99610dba89
Successfully built json2html


In [ ]:
import json
import osmnx as ox
import networkx as nx
import folium

# Load JSON data (example format provided)
with open('test_locations.json', 'r') as file:
    itinerary = json.load(file)

# Example JSON structure (itinerary.json):
# [{"name": "Attraction1", "lat": 1.3114, "lng": 103.8531}, ...]

# Extract attraction coordinates
locations = [(item['lat'], item['lng']) for item in itinerary]
names = [item['name'] for item in itinerary]

# Fetch road network data around the first attraction
# the network type can be 'drive', 'walk', 'bike', 'all', 'all_private', 'none'
G = ox.graph_from_point(locations[0], dist=5000, network_type="drive", simplify=True)

# Find nodes for all attractions
nodes = [ox.nearest_nodes(G, lng, lat) for lat, lng in locations]

# Calculate shortest paths between sequential attractions
routes = []
for i in range(len(nodes) - 1):
    route = nx.astar_path(G, nodes[i], nodes[i+1], weight="length")
    routes.append(route)

# Initialize Folium map centered on the first attraction
m = folium.Map(location=locations[0], zoom_start=14)

# Plot all routes on map
for route, name_start, name_end in zip(routes, names[:-1], names[1:]):
    coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
    folium.PolyLine(coords, color="blue", weight=5, opacity=0.8,
                    tooltip=f"{name_start} → {name_end}").add_to(m)

# Add markers for each attraction
for coord, name in zip(locations, names):
    folium.Marker(coord, popup=name,
                  icon=folium.Icon(color="red", icon="star")).add_to(m)

# Save map to HTML
m.save('an_itinerary_with_diff_places.html')

print("Map has been generated successfully!")


Map has been generated successfully!
